# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [28]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [29]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("~/scraping_repo/scraping/notebooks/books_infos.csv")
df_books.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jewao/scraping_repo/scraping/notebooks/books_infos.csv'

## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [ ]:
# Vérification des types de données
df_books.dtypes


Unnamed: 0       int64
title           object
price           object
rating          object
availability    object
dtype: object

Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [ ]:
# Conversion de title en chaîne de caractères
df_books["title"] = df_books["title"].astype("string")

# Vérification du type de la colonne title
print(df_books["title"].dtype)

df_books.dtypes

string


Unnamed: 0               int64
title           string[python]
price                   object
rating                  object
availability            object
dtype: object

Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [ ]:
# Convertir la colonne price en type décimal
df_books["price"] = df_books["price"].str.replace("£", " ").astype(float)

# Vérification du type de la colonne price
print(df_books["price"].head())

0    51.77
1    53.74
2    50.10
3    47.82
4    54.23
Name: price, dtype: float64


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [ ]:
# Valeurs possibles de la colonne availability (InStock/Out of stock)
print(df_books["availability"].head())

0    In stock
1    In stock
2    In stock
3    In stock
4    In stock
Name: availability, dtype: object


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [ ]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    if isinstance(value, bool):
        return value
    if isinstance(value, str):
        return "In stock" in value.strip()
    return False
    return "In stock" in value
convert_availability("In stock")

True

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [ ]:
# Convertir la colonne availability en booléen (True/False)
df_books["availability"] = df_books["availability"].apply(convert_availability)

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)

bool


Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [ ]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
ratings_map = ({
    "One": 1, 
    "Two": 2, 
    "Three": 3, 
    "Four": 4, 
    "Five": 5
})

df_books["rating"] = df_books["rating"].map(ratings_map)
# Vérification du type de la colonne rating
print(df_books.head())
print(df_books.dtypes)

   Unnamed: 0                         title  price  rating  availability
0           0            A Light in the ...  51.77       3          True
1           1            Tipping the Velvet  53.74       1          True
2           2                    Soumission  50.10       1          True
3           3                 Sharp Objects  47.82       4          True
4           4  Sapiens: A Brief History ...  54.23       5          True
Unnamed: 0               int64
title           string[python]
price                  float64
rating                   int64
availability              bool
dtype: object


In [ ]:
# # Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
    df_books["title"] = df_books["title"].astype("string")
    df_books["rating"] = df_books["rating"].map({"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}).astype("Int64")
    df_books["price"] = df_books["price"].astype(str).str.replace("£", "").astype(float)
    df_books["availability"] = df_books["availability"].apply(convert_availability)

    return df_books

convert_types(df_books).dtypes

Unnamed: 0               int64
title           string[python]
price                  float64
rating                   Int64
availability              bool
dtype: object

---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [ ]:
# Création de la BDD et insertion des données
connection = sqlite3.connect("book_store.db")
df_books.to_sql('books', connection)

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [ ]:
# Compter le nombre de livre dans la BDD
cursor = connection.cursor()
cursor.execute("SELECT COUNT(*) FROM books")
rows = cursor.fetchone()[0]
print(rows)

1000
